**Quandl API**

In [22]:
# # import needed libraries
# import quandl
# import pandas as pd

# # add quandl API key for unrestricted
# quandl.ApiConfig.api_key = 'UstfyTzdVRpjmScPq7yn'

# # get the table for daily stock prices and,
# # filter the table for selected tickers, columns within a time range
# # set paginate to True because Quandl limits tables API to 10,000 rows per call
# data = quandl.get_table('WIKI/PRICES', ticker = ['ACIA','ACMR'], 
#                         qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
#                         date = { 'gte': '2005-01-01', 'lte': '2018-12-31' }, 
#                         paginate=True)

# # create a new dataframe with 'date' column as index
# new = data.set_index('date')

# # use pandas pivot function to sort adj_close by tickers
# clean_data = new.pivot(columns='ticker')

**Zipline Quandl Bundle**

In [14]:
import os
import pandas as pd
import numpy as np
import re
from pandas import Timestamp
from zipline.data.bundles import load
from zipline.data.bundles.quandl import quandl_bundle
from zipline.data.data_portal import DataPortal
from zipline.utils.calendars import get_calendar

In [23]:
now = Timestamp.utcnow()
bundle = load('quandl', os.environ, now)

all_assets = bundle.asset_finder.retrieve_all(bundle.asset_finder.sids)
symbols = set(
str(asset.symbol) for asset in bundle.asset_finder.retrieve_all(bundle.asset_finder.equities_sids)
)

quandl_data = DataPortal(asset_finder= bundle.asset_finder,
trading_calendar = get_calendar('NYSE'),
first_trading_day = bundle.equity_daily_bar_reader.first_trading_day,
equity_minute_reader=bundle.equity_minute_bar_reader,
equity_daily_reader=bundle.equity_daily_bar_reader,
adjustment_reader=bundle.adjustment_reader)

end_dt = pd.Timestamp('2017-12-31', tz = 'utc')

In [16]:
available_files = !ls -ltS ../intrinio/financials/requests | awk '{if($5 > 1000) print $9}'
available_tickers = np.unique([re.match('([A-Z]+)_.*', f).group(1) for f in available_files])

In [17]:
set(available_tickers) - symbols

{'ACIA',
 'ACMR',
 'ALRM',
 'AMN',
 'ANET',
 'APPF',
 'APPN',
 'APTI',
 'AQ',
 'AR',
 'ARCH',
 'ARLP',
 'ASUR',
 'AXTI',
 'AYX',
 'BAND',
 'BGSF',
 'BKI',
 'BL',
 'BOX',
 'BPL',
 'BSM',
 'BWXT',
 'CAI',
 'CARG',
 'CARS',
 'CDEV',
 'CLB',
 'CLDR',
 'COMM',
 'CP',
 'CRC',
 'CRTO',
 'CVRR',
 'DNOW',
 'DVMT',
 'DWCH',
 'ECA',
 'EEP',
 'ENB',
 'ETP',
 'EXTN',
 'FEYE',
 'FI',
 'FIT',
 'FLEX',
 'FLOW',
 'FRAC',
 'FSCT',
 'FSNN',
 'GDDY',
 'GDI',
 'GEL',
 'GTE',
 'HDP',
 'HEP',
 'HESM',
 'HUBS',
 'IMO',
 'INOV',
 'INSE',
 'INST',
 'IOTS',
 'KE',
 'KELYB',
 'KLIC',
 'LOGI',
 'LTRPA',
 'LTRPB',
 'MB',
 'MCRN',
 'MDB',
 'MIC',
 'MIME',
 'MITL',
 'MJCO',
 'MLNX',
 'MMP',
 'MOBL',
 'MPLX',
 'MRAM',
 'MTCH',
 'MX',
 'NBLX',
 'NEWR',
 'NGL',
 'NH',
 'NS',
 'NTNX',
 'OKTA',
 'ON',
 'OOMA',
 'OTEX',
 'PAA',
 'PAGP',
 'PSDO',
 'PSTG',
 'PSXP',
 'PTC',
 'PUMP',
 'PXLW',
 'QADB',
 'QTNA',
 'RPD',
 'SABR',
 'SCWX',
 'SEDG',
 'SGH',
 'SHLX',
 'SLP',
 'SNAP',
 'SNDR',
 'SOHU',
 'SPXC',
 'SQ',
 'SRCI',
 'SRT'

In [24]:
quandl_symbols = []
for ticker in symbols:
    try:
        quandl_symbols.append(quandl_data.asset_finder.lookup_symbol(ticker,end_dt))
    except:
        continue
quandl_pricing = quandl_data.get_history_window(quandl_symbols,end_dt,1000,'1d','close','daily')

In [26]:
quandl_pricing.shape

(1000, 3196)